In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from jcc2_data_processor import create_processor
import pandas as pd

In [ ]:
# Load data
data_file = '/home/nathanjgaul/Downloads/DCDC UQ Combined_2025-07-30_pii_scrubbed.csv'
processor = create_processor(data_file)
df = processor.load_data()

# Check if any app-specific effectiveness fields exist
print("Checking for app-specific effectiveness fields...\n")

# Look for fields that match pattern: section.something_effectiveness_appname
apps = ['a2it', 'cad', 'codex', 'crucible', 'cyber9line', 'dispatch',
        'jcc2cyberops', 'jcc2readiness', 'madss', 'rally', 'redmap',
        'sigact', 'threathub', 'triage', 'unity']

sections = ['mop_1_1_1', 'mos_1_1_2', 'reporting_and_data_export']

app_fields_found = {}

for section in sections:
    print(f"\nSection: {section}")
    print("=" * 60)
    
    # Get all fields for this section
    section_fields = [col for col in df.columns if col.startswith(f"{section}.")]
    
    # Filter for effectiveness/suitability fields
    rating_fields = [f for f in section_fields if any(x in f for x in ['effectiveness', 'suitability'])]
    
    print(f"Total rating fields: {len(rating_fields)}")
    
    # Check each app
    for app in apps:
        # Find fields ending with this app name
        app_specific = [f for f in rating_fields if f.endswith(app) or f.endswith(f"_{app}")]
        
        if app_specific:
            print(f"\n  {app.upper()}: {len(app_specific)} fields found")
            for field in app_specific[:2]:  # Show first 2
                # Check if there's actual data
                non_null = df[field].notna().sum()
                print(f"    - {field}")
                print(f"      Non-null values: {non_null}")
                if non_null > 0:
                    # Show value distribution
                    value_counts = df[field].value_counts().head(3)
                    print(f"      Top values: {value_counts.to_dict()}")
    
    # Also check for overall fields
    overall_fields = [f for f in rating_fields if 'overall' in f]
    if overall_fields:
        print(f"\n  OVERALL: {len(overall_fields)} fields found")
        for field in overall_fields[:2]:
            non_null = df[field].notna().sum()
            print(f"    - {field}")
            print(f"      Non-null values: {non_null}")